<a href="https://colab.research.google.com/github/ua-datalab/Geospatial_Workshops/blob/main/notebooks/STAC_crawl2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install pystac_client. This library is used to crawl SpatioTemporal Asset Catalogs (STAC)
!pip install pystac_client --quiet
!pip install planetary_computer --quiet

In [ ]:
##Install library to work with raster images

!pip install rasterio --quiet

In [ ]:
!pip install rioxarray

In [14]:
import rasterio
from rasterio.plot import show
import matplotlib.pyplot as plt
import requests
import numpy as np

In [2]:
import pystac_client
import planetary_computer

catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

In [ ]:
#from pystac_client import Client
collections = list(catalog.get_collections())

# Print the number of collections
print(f"Number of collections in the base catalog: {len(collections)}")

In [ ]:
time_range = "2020-12-01/2020-12-31"
bbox = [-122.2751, 47.5469, -121.9613, 47.7458]

search = catalog.search(collections=["landsat-c2-l2"], bbox=bbox, datetime=time_range)
items = search.get_all_items()
len(items)



In [ ]:
for item in items:
    print(f"Item ID: {item.id}")

    # Print the asset names (keys) for each item
    print("Assets:")
    for asset_key in item.assets:
        print(f"  - {asset_key}: {item.assets[asset_key].href}")
    print("---------------------------------------------------")

In [ ]:
import geopandas

df = geopandas.GeoDataFrame.from_features(items.to_dict(), crs="epsg:4326")
df

In [ ]:
selected_item = min(items, key=lambda item: item.properties["eo:cloud_cover"])
print(selected_item)

In [ ]:
import rich.table

table = rich.table.Table("Asset Key", "Description")
for asset_key, asset in selected_item.assets.items():
    table.add_row(asset_key, asset.title)

table

In [7]:
selected_item.assets["rendered_preview"].to_dict()

{'href': 'https://planetarycomputer.microsoft.com/api/data/v1/item/preview.png?collection=landsat-c2-l2&item=LC08_L2SP_047027_20201204_02_T1&assets=red&assets=green&assets=blue&color_formula=gamma+RGB+2.7%2C+saturation+1.5%2C+sigmoidal+RGB+15+0.55&format=png',
 'type': 'image/png',
 'title': 'Rendered preview',
 'rel': 'preview',
 'roles': ['overview']}

In [ ]:
from IPython.display import Image

Image(url=selected_item.assets["rendered_preview"].href, width=500)

In [ ]:
selected_item.assets["blue"].href[:250]

In [ ]:
import requests

requests.head(selected_item.assets["blue"].href).status_code

In [ ]:
# import xarray as xr
import rioxarray

ds = rioxarray.open_rasterio(
    selected_item.assets["blue"].href, overview_level=4
).squeeze()
img = ds.plot(cmap="Blues", add_colorbar=False)
img.axes.set_axis_off();

In [ ]:
ds

In [ ]:
img

In [ ]:
##Retrieve the file size of the COG

# Send an HTTP HEAD request
response = requests.head(selected_item.assets["blue"].href)

if response.status_code == 200:
    # Check if 'Content-Length' header is available
    if 'Content-Length' in response.headers:
        file_size_bytes = int(response.headers['Content-Length'])
        print(f"File Size: {file_size_bytes} bytes")
    else:
        print("Content-Length header is not available.")
else:
    print(f"Failed to access the file. Status code: {response.status_code}")